In [47]:
import json
import os
import numpy as np

In [48]:
os.chdir('/Users/Yanean/Desktop/ML/HW2/cooking')
train = json.loads(open('train.json').read())
test = json.loads(open('test.json').read())

In [49]:
cat = set()
lbl = set()
sample = set()

for k in train:
    ing = k['ingredients']
    for u in ing:
        cat.add(u)
    lbl.add(k['cuisine'])
    sample.add(k['id'])

num_of_ingredients = len(cat)
num_of_cuisine = len(lbl)
num_of_sample = len(sample)
print(num_of_cuisine)
print(num_of_ingredients)
print(len(sample))

cat_map = {}
for i,t in enumerate(list(cat)):
    cat_map[t] = i
lbl_map = {}
for i,t in enumerate(list(lbl)):
    lbl_map[t] = i
print (lbl_map)

20
6714
39774
{'cajun_creole': 0, 'indian': 1, 'japanese': 2, 'chinese': 3, 'brazilian': 4, 'italian': 5, 'french': 6, 'korean': 7, 'vietnamese': 8, 'filipino': 9, 'greek': 10, 'mexican': 11, 'irish': 12, 'spanish': 13, 'moroccan': 14, 'british': 15, 'jamaican': 16, 'southern_us': 17, 'russian': 18, 'thai': 19}


In [56]:
train_data = []
train_label = []
features  = []
for k in train:
    ing = k['ingredients']
    features = [0]*num_of_ingredients
    for u in ing:
        features[cat_map[u]] = 1
    train_data.append(features)
    train_label.append(lbl_map[k['cuisine']])


train_data, train_label = np.array(train_data), np.array(train_label)
train_data.shape, train_label.shape
print(train_label)
print(train_data)

[0 0 0 ..., 0 0 0]


In [25]:
print(test[0]['ingredients'])

['baking powder', 'eggs', 'all-purpose flour', 'raisins', 'milk', 'white sugar']


In [28]:
test_data = []
cat1 = set()

for a in test:
    ing = a['ingredients']
    for b in ing:
        cat1.add(b) 
number_of_ingredient = len(cat) 
    
for a in test:
    ing = a['ingredients'] 
    features = [0]*number_of_ingredient # use the same length of the list in train data set
    for b in ing:
        if b not in cat_map.keys():
            continue
        features[cat_map[b]] = 1
    test_data.append(features)
test_data = np.array(test_data)

In [29]:
from sklearn import cross_validation
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
clf1 = GaussianNB()
cv1 = cross_validation.KFold(len(train_data),n_folds = 3)

for tr_cv,te_cv in cv1:
    clf1.fit(train_data[tr_cv],train_label[tr_cv])
    pred1 = clf1.predict(train_data[te_cv])
    accuracy1 = accuracy_score(pred1,train_label[te_cv])
    print(accuracy1)

0.379016442902
0.382938603108
0.377583345904


In [30]:
from sklearn.naive_bayes import BernoulliNB
clf2 = BernoulliNB()
for tr_cv,te_cv in cv1:
    clf2.fit(train_data[tr_cv],train_label[tr_cv])
    pred2 = clf2.predict(train_data[te_cv])
    accuracy2 = accuracy_score(pred2,train_label[te_cv])
    print(accuracy2)

0.684190677327
0.679514255544
0.686906019007


In [31]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data, train_label)
from sklearn import metrics, cross_validation
pred3 = cross_validation.cross_val_predict(logreg, train_data, train_label,cv=3)
accuracy3 = metrics.accuracy_score(train_label,pred3)
print(accuracy3)


0.775758032886


In [32]:
logreg.fit(train_data, train_label)
result = logreg.predict(test_data)

In [46]:
import pandas as pd
test = pd.read_json('test.json')

inv_map = {}
for i in lbl_map:
    inv_map[lbl_map[i]]=i
temp = []
for i in result:
    temp.append(inv_map[i]);

test['cuisine'] = temp
test = test.drop('ingredients', axis = 1)
test.to_csv('output.csv')
